In [0]:
!pip install tensorflow==2.0.0-beta1 --upgrade
!pip install tensorflow-gpu==2.0.0-beta1 --upgrade

In [0]:
from datetime import datetime
import math
import os
import random
import shutil
import sys
import time
import zipfile

from IPython.display import clear_output

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print('Tensorflow Version: ', tf.__version__)
print('GPU: ', tf.test.gpu_device_name())
from tensorflow.keras import Sequential, layers, models, Input, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import files
os.listdir('.')

In [0]:
upload_new = False

images_train_dirname = os.path.join('train')
images_validation_dirname = os.path.join('validation')


def delete_warning():
  for f in os.listdir('.'):
    if 'train' in f or 'valid' in f:
      for i in range(10, 0, -1):
        cancel_msg = 'Go to "Runtime -> Interrupt execution" to cancel'
        print('Erasing data in {0} seconds. {1}'.format(i, cancel_msg))
        time.sleep(1)
      return


def clear_directory():
  delete_warning()
  for f in os.listdir('.'):
    if 'train' in f or 'valid' in f or 'scene' in f:
      print('Deleting', f)
      if 'zip' in f:
        os.remove(f)
      else:
        shutil.rmtree(f)


def parse_examples(location, overwrite=False):
  print(location)
  if overwrite and os.path.isdir(location):
    print('\tErasing directory')
    shutil.rmtree(location)
  if not os.path.isdir(location):
    print('\tUnzipping')
    zip_ref = zipfile.ZipFile(location + '.zip', 'r')
    zip_ref.extractall(os.path.dirname(location))
    zip_ref.close()
  img_dir = os.listdir(os.path.join(location, 'face'))
  print('\tFound', len(img_dir), 'examples')
  return len(img_dir)


if upload_new:
  clear_directory()
  files.upload()
total_train_examples = parse_examples(
  images_train_dirname, overwrite=True)
total_validation_examples = parse_examples(
  images_validation_dirname, overwrite=True)

In [0]:
img_size = 160


def create_train_generators(batch_size=1, shuffle=True):
  img_options = {
    'rescale': 1 / 255
  }
  img_dir_options = {
    'target_size': (img_size, img_size),
    'batch_size': batch_size,
    'class_mode': 'input',
    'shuffle': shuffle,
    'classes': ['face']
  }
  # block print statements
  old_stdout = sys.stdout
  sys.stdout = open(os.devnull, 'w')
  train_datagen = ImageDataGenerator(**img_options).flow_from_directory(
    images_train_dirname, **img_dir_options)
  validation_datagen = ImageDataGenerator(**img_options).flow_from_directory(
    images_validation_dirname, **img_dir_options)
  # reenable print statements
  sys.stdout = old_stdout
  return train_datagen, validation_datagen

In [0]:
def create_encoder_decoder(latent_units=128,
                           conv_filters=[32, 64, 128, 256],
                           dense_units=[],
                           latent_activation=None,
                           final_activation=None,
                           show_summary=False,
                           **kwargs):
  '''
    ENCODER model
  '''
  enc_conv_settings = {
    'kernel_size': [3, 3],
    'padding': 'valid',
    'use_bias': True,
    'activation': None
  }
  enc_relu_settings = {
    'max_value': None,
    'negative_slope': 0.0
  }
  # INPUT
  encoder = Sequential()
  # CONV layers
  for i, f in enumerate(conv_filters):
    if i == 0:
      encoder.add(layers.Conv2D(
        f, input_shape=[img_size, img_size, 3], **enc_conv_settings))
    else:
      encoder.add(layers.Conv2D(f, **enc_conv_settings))
    encoder.add(layers.BatchNormalization())
    encoder.add(layers.ReLU(**enc_relu_settings))
    encoder.add(layers.MaxPool2D())
  # hidden DENSE layers
  encoder.add(layers.Flatten())
  flattened_units = encoder.output_shape[-1]  # save flattened shape for decoder
  for u in dense_units:
    encoder.add(layers.Dense(u, activation='relu'))
  # LATENT space
  encoder.add(layers.Dense(latent_units, activation=latent_activation))
  if show_summary:
    encoder.summary()
  '''
    DECODER model
  '''
  dec_dense_units = dense_units[::-1]
  dec_conv_filters = conv_filters[::-1]
  hidden_conv_dim = int(img_size / (2 ** len(dec_conv_filters)))
  pre_conv_depth = int(flattened_units / (hidden_conv_dim * hidden_conv_dim))
  dec_conv_reshape = [hidden_conv_dim, hidden_conv_dim, pre_conv_depth]
  dec_hidden_units = dec_conv_reshape[0] * dec_conv_reshape[1] * dec_conv_reshape[2]
  dec_conv_settings = enc_conv_settings.copy()
  dec_conv_settings.update(
      {'padding': 'same', 'activation': 'relu', 'strides': [2, 2]})
  final_conv_settings = dec_conv_settings.copy()
  final_conv_settings.update({'activation': final_activation, 'strides': [1, 1]})
  # INPUT
  decoder = Sequential()
  # hidden DENSE layers
  for i, u in enumerate(dec_dense_units):
    if i == 0:
      decoder.add(layers.Dense(
        u, input_shape=[latent_units], activation='relu'))
    else:
      decoder.add(layers.Dense(u, activation='relu'))
  # CONV layers
  if len(dec_dense_units) == 0:
    decoder.add(layers.Dense(
      dec_hidden_units, input_shape=[latent_units], activation='relu'))
  else:
    decoder.add(layers.Dense(dec_hidden_units, activation='relu'))
  decoder.add(layers.Reshape(dec_conv_reshape))
  for f in dec_conv_filters:
    decoder.add(layers.Conv2DTranspose(f, **dec_conv_settings))
  # final CONV layer
  decoder.add(layers.Conv2DTranspose(3, **final_conv_settings))
  if show_summary:
    decoder.summary()
  return encoder, decoder

In [0]:
# create global variable for saving histories
# but only if it has not been defined yet
try:
  saved_histories
except NameError:
  for i in range(3, 0, -1):
    print('Creating new saved_histories in {0} seconds'.format(i))
    time.sleep(1)
  saved_histories = []


def plot_history(hist, ignore=0):
  if hasattr(hist, 'history'):
    hist = hist.history
  epochs = list(range(len(hist['loss'])))
  ignore = max(min(ignore, len(hist['loss']) - ignore), 0)
  for k in hist.keys():
    if 'val_' in k:
      continue
    if 'lr' in k:
      plt.plot(hist[k][ignore:], hist['loss'][ignore:], label='lr & Loss')
    else:
      plt.plot(
        epochs[ignore:], hist[k][ignore:], label='Train {0}'.format(k))
      plt.plot(
        epochs[ignore:], hist['val_' + k][ignore:], label='Valid {0}'.format(k))
    plt.legend()
    plt.show()


def plot_saved_histories(hists, key, p_label=None, ignore=0):
  if len(hists) == 0:
    return
  for i, hist in enumerate(hists):
    h = hist['history']
    if hasattr(h, 'history'):
      h = h.history
    label = '[{0}]'.format(i)
    if p_label:
      l = hist['params'][p_label]
      if l is None:
        l = 'None'
      label = '{0} {1}'.format(label, l)
    epochs = range(ignore, len(h[key]))
    plt.plot(epochs, h[key][ignore:], label=label)
  plt.legend()
  plt.show()
  for i, hist in enumerate(hists):
    h = hist['history']
    if hasattr(h, 'history'):
      h = h.history
    lbl = i
    if p_label:
      lbl = hist['params'][p_label]
    print(lbl, '--> {:.5E}'.format(min(h[key])))

In [0]:
class DisplayAutoencoderCallback(tf.keras.callbacks.Callback):

  def __init__(self, model, generator, test_step=1):
    tf.keras.callbacks.Callback.__init__(self)
    self.model = model
    self.generator = generator
    self.test_step = test_step

  def on_epoch_begin(self, epoch, logs=None):
    if epoch == 0:
      draw_random_prediction(self.model, gen=self.generator)

  def on_epoch_end(self, epoch, logs=None):
    if epoch % self.test_step == 0:
      draw_random_prediction(self.model, gen=self.generator)

In [0]:
def build_and_test_model(graph_label=None,
                         epochs=100,
                         batch_size=128,
                         show_summary=False,
                         **kwargs):
  print('####################')
  print('Appending to {0} Saved Histories'.format(len(saved_histories)))
  model_params = {
    'latent_units': kwargs.get('latent_units', 128),
    'conv_filters': kwargs.get('conv_filters', [32, 64, 128, 256]),
    'dense_units': kwargs.get('dense_units', []),
    'latent_activation': kwargs.get('latent_activation', None),
    'final_activation': kwargs.get('final_activation', None),
    'learning_rate': kwargs.get('learning_rate', 0.001),
    'epsilon': kwargs.get('epsilon', 1e-7),
    'batch_size': batch_size
  }
  for key, val in model_params.items():
    arrow = ''
    if graph_label and key == graph_label:
      arrow = '<======='
    print('  -', key, ':', val, arrow)
  if graph_label:
    assert graph_label in model_params
  ''' Build the Model(s) '''
  encoder, decoder = create_encoder_decoder(
    show_summary=show_summary, **model_params)
  if show_summary:
    return
  model_autoencoder = Sequential([encoder, decoder])
  ''' Train/Validation Generator '''
  train_gen, val_gen = create_train_generators(batch_size, shuffle=True)
  ''' Callbacks during training '''
  progress = tf.keras.utils.Progbar(epochs, unit_name='Epochs')
  callbacks = [
  #   DisplayAutoencoderCallback(model_autoencoder, val_gen, test_step=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
  ]
  if graph_label:
    callbacks.append(tf.keras.callbacks.LambdaCallback(
      on_epoch_begin=lambda epoch, logs: progress.update(epoch),
      on_train_end=lambda logs: progress.update(epochs)
    ))
  ''' Compile '''
  model_autoencoder.compile(
    optimizer=tf.keras.optimizers.Adam(
      learning_rate=model_params['learning_rate'],
      epsilon=model_params['epsilon']
    ),
    loss='mse'
  )
  ''' Train '''
  print('Will exit training once "val_loss" no longer decreases')
  verbose = 1
  if graph_label:
    verbose = 0
  history = model_autoencoder.fit_generator(
    train_gen,
    steps_per_epoch=int(total_train_examples / batch_size),
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=int(total_train_examples / batch_size),
    verbose=verbose,
    callbacks=callbacks
  )
  print()
  ''' Save History and Params '''
  saved_histories.append({
    'history': history.history,
    'params': model_params,
    'model': model_autoencoder,
    'encoder': encoder,
    'decoder': decoder
  })
  if graph_label:
    plot_saved_histories(
      saved_histories, 'val_loss', p_label=graph_label, ignore=10)
  else:
    plot_history(saved_histories[-1]['history'], ignore=10)

In [0]:
build_and_test_model(
  batch_size=32,
  learning_rate=0.00001,
  epochs=1000
)

# for b in [16, 32, 64, 128, 256]:
#   build_and_test_model(
#     graph_label='batch_size',
#     batch_size=32,
#     learning_rate=0.0001,
#     epochs=1000
#   )

# for lr in [0.005, 0.001, 0.0001, 0.00001]:
#   build_and_test_model(
#     graph_label='learning_rate',
#     batch_size=32,
#     learning_rate=lr,
#     epochs=1000
#   )

# for units in [8, 16, 32, 64, 128, 256]:
#   build_and_test_model(
#     graph_label='latent_units',
#     latent_units=units,
#     batch_size=32,
#     learning_rate=0.001,
#     epochs=1000
#   )

In [0]:
drive_model_folder = os.path.join('models')
model_file_name = 'simpsons_model_{0}'

if not os.path.exists(drive_model_folder):
    os.makedirs(drive_model_folder)

def dated_filename():
  return str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S'))


def save_model(model, name):
  blob_name = '{0}_{1}.h5'.format(name, dated_filename())
  print('Saving -> {}'.format(blob_name))
  file_name = os.path.join(drive_model_folder, blob_name)
  model.save(file_name)
  return file_name


def load_latest_model(name, custom_objects={}):
  model_files = [
      f
      for f in os.listdir(drive_model_folder)
      if name in f and 'h5' in f
  ]
  model_files.sort()
  latest_file_name = model_files[-1]
  latest_filepath = os.path.join(drive_model_folder, latest_file_name)
  print('Loading -> {}'.format(latest_filepath))
  model = tf.keras.models.load_model(
      latest_filepath, custom_objects=custom_objects)
  return model

In [0]:
for hist in saved_histories:
  b_size = hist['params']['batch_size']
  enc_path = save_model(hist['encoder'], 'simpsons_ae_encoder_{0}'.format(b_size))
  dec_path = save_model(hist['decoder'], 'simpsons_ae_decoder_{0}'.format(b_size))

In [0]:
def draw_random_prediction(model, count=1, gen=None):
  all_images = []
  if gen is None:
    _, gen = create_train_generators()
  for i in range(count):
    x, _ = next(gen)
    all_images.append(x[0])
    guess = model.predict(x)
    guess_img = np.clip(guess[0], 0, 1)  # constrain to be 0-1
    all_images.append(guess_img)
  show_images(all_images, cols=count)


def show_images(images, cols=1):
    n_images = len(images)
    fig = plt.figure()
    for n, image in enumerate(images):
      index = int(n / 2)
      if n % 2 == 1:
        index += int(n_images / 2)
      a = fig.add_subplot(2, cols, index + 1)
      plt.imshow(image)
    size = 3
    fig.set_size_inches(int(size * cols * 0.5), size)
    plt.show()

In [0]:
for hist in saved_histories:
  print(hist['params'])
  draw_random_prediction(hist['model'], count=10)

In [0]:
print(os.path.getsize(enc_path))
files.download(enc_path)
print(os.path.getsize(dec_path))
files.download(dec_path)

In [0]:
upload_model_h5 = False

if upload_model_h5:
  files.upload()
  for f in os.listdir():
    if '.h5' in f:
      shutil.move(f, os.path.join(drive_model_folder, f))

In [0]:
load_prebuilt_model = False

if load_prebuilt_model:
  encoder = load_latest_model('simpsons_ae_encoder')
  decoder = load_latest_model('simpsons_ae_decoder')
  model_autoencoder = Sequential([encoder, decoder])

In [0]:
saved_histories = []
tf.keras.backend.clear_session()